In [1]:
import pandas as pd
import os
import json
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin, unquote

# URL of the page containing the link
url = "https://www.countyhealthrankings.org/explore-health-rankings/georgia/data-and-resources"

# Sending a GET request to the webpage
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Finding and downloading Excel files
for link in soup.find_all('a'):
    href = link.get('href')
    #print("href is: " + href)
    if href and (href.endswith('.xlsx') or href.endswith('.xls')):
        # Sending a GET request to download the Excel file
        full_url = urljoin(url, href)
        file_response = requests.get(full_url)
        
        # Saving the file
        file_name = unquote(os.path.basename(full_url))
        
        # Ensure the directory exists
        os.makedirs('./County_Health_Rankings_Data/', exist_ok=True)
        
        # Save the content in the specified directory
        with open(f"./County_Health_Rankings_Data/{file_name}", 'wb') as file:
            file.write(file_response.content)
        
        print(f"Downloaded file: {file_name}")


Downloaded file: 2023 County Health Rankings Georgia Data - v3.xlsx
Downloaded file: 2022 County Health Rankings Georgia Data - v2.xlsx
Downloaded file: 2021 County Health Rankings Georgia Data - v1.xlsx
Downloaded file: 2020 County Health Rankings Georgia Data - v1_0.xlsx
Downloaded file: 2019 County Health Rankings Georgia Data - v1_0.xls
Downloaded file: 2018 County Health Rankings Georgia Data - v3.xls
Downloaded file: 2017 County Health Rankings Georgia Data - v2.xls
Downloaded file: 2016 County Health Rankings Georgia Data - v3.xls
Downloaded file: 2015 County Health Rankings Georgia Data - v3.xls
Downloaded file: 2014 County Health Rankings Georgia Data - v6.xls
Downloaded file: 2013 County Health Ranking Georgia Data - v1_0.xls
Downloaded file: 2012 County Health Ranking Georgia Data - v4.xls
Downloaded file: 2011 County Health Ranking Georgia Data - v4.xls
Downloaded file: 2010 County Health Ranking Georgia Data - v2.xls


In [8]:
def process_data(data_dict, full_path):
    try:
        ranked_measure_data = pd.read_excel(full_path, sheet_name="Ranked Measure Data", header=1)
        additional_measure_data = pd.read_excel(full_path, sheet_name="Additional Measure Data", header=1)
    except Exception as e:
        print(f"Error processing file: {full_path}")
        print(e)
        return
        
    #1. Extract information from the Ranked Measure Data
    
    # Extract correct column names for ranked measure data
    ranked_measure_data_cols = ranked_measure_data.columns.to_list()

    # Iterate over each row to populate the dictionary
    for i, row in ranked_measure_data.iterrows():
        if i < 2:  # Skip the metadata rows
            continue
        
        # Extract the county name
        county = row['County']  # Extract using the correct column name
        if pd.isna(county):  # Skip rows where county name is NaN
            continue
        
        # Initialize a dictionary for the county
        data_dict[county] = {}
        
        # Iterate over each variable and extract the valid average value and its corresponding 95% confidence interval
        variable = None
        for idx in range(3, len(ranked_measure_data_cols) - 1):  # Start iterating from the 4th column and leave room for CI columns
            col = ranked_measure_data_cols[idx]
            value = row[col]
            
            # Check if the column is a valid variable with 'Rate' or 'Ratio' and has both 95% CI columns
            if ('Rate' in col or 'Ratio' in col) and ('CI' not in col) and '95% CI' in ranked_measure_data_cols[idx + 1] and '95% CI' in ranked_measure_data_cols[idx + 2]:
                variable = col
                average_value = value
                ci_low = row[ranked_measure_data_cols[idx + 1]]
                ci_high = row[ranked_measure_data_cols[idx + 2]]

                # Check for NaN values in all three attributes before adding to the dictionary
                if (not pd.isna(average_value) and not pd.isna(ci_low) and not pd.isna(ci_high)):
                    data_dict[county][variable] = {'average': average_value,
                                                '95% CI - Low': ci_low,
                                                '95% CI - High': ci_high}
                    
    # 2. Extract information from the Additional Measure Data
    
    additional_measure_data_cols = additional_measure_data.columns.tolist()

    for i, row in additional_measure_data.iterrows():
        if i < 2: continue 
        county = row['County']

        if pd.isna(county) or county not in data_dict.keys():
            continue

        for idx in range(3, len(additional_measure_data_cols)-1):  # Use the correct columns list
            col = additional_measure_data_cols[idx]  # Use the correct columns list
            average_value = row[col]
            
            # Ensure we have the '95% CI' columns next
            if ('Rate' in col or 'Ratio' in col) and ('CI' not in col) and '95% CI' in additional_measure_data_cols[idx + 1] and '95% CI' in additional_measure_data_cols[idx + 2]:
                ci_low = row[additional_measure_data_cols[idx + 1]]
                ci_high = row[additional_measure_data_cols[idx + 2]]
                
                # Check that all three values are not NaN
                if not pd.isna(average_value) and not pd.isna(ci_low) and not pd.isna(ci_high):
                    data_dict[county][col] = {
                        'average': average_value,
                        '95% CI - Low': ci_low,
                        '95% CI - High': ci_high
                    }


            

In [4]:
def setup_database():
    import sqlite3
    conn = sqlite3.connect('health_rankings.db')
    c = conn.cursor()

    #Create Table
    c.execute('''
    CREATE TABLE IF NOT EXISTS Counties (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL UNIQUE
    )
    ''')

    c.execute('''
    CREATE TABLE IF NOT EXISTS Years (
        id INTEGER PRIMARY KEY,
        year INTEGER NOT NULL UNIQUE
    )
    ''')

    c.execute('''
    CREATE TABLE IF NOT EXISTS Metrics (
        id INTEGER PRIMARY KEY,
        county_id INTEGER, 
        year_id INTEGER,
        variable TEXT NOT NULL,
        average REAL,
        ci_low REAL,
        ci_high REAL,
        FOREIGN KEY (county_id) REFERENCES Counties (id),
        FOREIGN KEY (year_id) REFERENCES Years(id)    
    )
    ''')

    conn.commit()
    conn.close()


In [5]:
def insert_data_into_database(year, data_dict):
    conn = sqlite3.connect('health_rankings.db')
    c = conn.cursor()
    
    # Inserting counties and years 
    for county in data_dict.keys():
        c.execute("INSERT OR IGNORE INTO Counties (name) VALUES (?)", (county,))
        c.execute("INSERT OR IGNORE INTO Years (year) VALUES (?)", (year,))
    
    # Inserting metrics
    for county, metrics in data_dict.items():
        for variable, values in metrics.items():
            c.execute('''
            INSERT INTO Metrics (county_id, year_id, variable, average, ci_low, ci_high) 
            VALUES ((SELECT id FROM Counties WHERE name = ?),
                    (SELECT id FROM Years WHERE year = ?),
                    ?, ?, ?, ?)
            ''', (county, year, variable, values['average'], values['95% CI - Low'], values['95% CI - High']))
    
    conn.commit()
    conn.close()

In [6]:
def export_database():
    # Connect to your database
    conn = sqlite3.connect('health_rankings.db')

    # Query to fetch data
    #First line of query is defining what the column names will appear as in the output
    #Join command doesn't actually modify data in database. It temporarily associates (or maps)
    # the metrics table ids with their respective values for the purpose of viewing or analyzing 
    # in the query result
    query = """
    SELECT c.name as County, y.year as Year, m.variable as Variable, m.average as Average, m.ci_low as CI_Low, m.ci_high as CI_High
    FROM Metrics m
    JOIN Counties c ON m.county_id = c.id
    JOIN Years y ON m.year_id = y.id
    """

    # Load the data into a pandas DataFrame
    data = pd.read_sql(query, conn)

    # Save the data to a CSV file
    data.to_csv('full_database_export.csv', index=False)

    # Close the connection
    conn.close()


In [9]:
if __name__ == "__main__":
    setup_database()
    folder_name = "County_Health_Rankings_Data"
    
    for filename in os.listdir(folder_name):
        full_path = os.path.join(folder_name, filename)
        year = filename.split()[0] #Extract year from filename
        
        # Initialize a dictionary to store the data
        data_dict = {}
        
        # Process the data
        process_data(data_dict, full_path)
        
        # Insert data into database
        insert_data_into_database(year, data_dict)
        
        # Export database
        export_database()
        

Error processing file: County_Health_Rankings_Data/2010 County Health Ranking Georgia Data - v2.xls
Worksheet named 'Ranked Measure Data' not found
